1. Utilizando Python e Numpy, implemente uma Multi-layer Perceptron (MLP) com uma camada oculta de largura 2 (i.e., uma Perceptron com duas unidades ocultas), que aproxima função “ou exclusivo” (XOR) definida como XOR : {0, 1}² → {0, 1}.

In [1]:
import numpy as np

2. Considere que apenas a camada oculta tem uma função de ativação (adote ReLU) (i.e., a camada de saída tem
ativação identidade). Otimize a MLP com uma função de perda que penaliza o quadrado do erro de estimação.

In [ ]:
def ReLU(x):
    return max(0, x)

def I(x):
    return x